In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import accuracy, f1_score


In [2]:
horrorDF = pd.read_excel(
    "2조 괴담 파일.xlsx", skiprows=0, header=1, sheet_name="SY", usecols=[3]
)


In [3]:
horrorDF


,TEXT
0,오늘 학교에서 모의고사보는데 가위눌리다가 한쪽눈만 떠진상태로 교실에서 내 다리에 올...
1,그녀는 날 사랑한다... 안 사랑한다... 날 사랑한다... 안 사랑한다...날 사...
2,"어린애같지만, 난 항상 지하실에서 나갈때 내 뒤를 쳐다보면서 계단을 뛰어 올라가. ..."
3,생리혈에 덩어리 같은게 있을수 있다는건 나도 알아. 근데 그게 보통 움직이는거야?\n
4,"난 별 생각없이 ""우리 모두 다함께 손뼉쳐!"" 라고 외쳤어. 다락방에서 박수 소리를..."
...,...
994,희망봉근해에 출몰하는 네덜란드 동인도회사의 유령선을 플라잉더치맨이라고 부른다. 알류...
995,"T 씨가 잠에서 깨자, 1년이 경과해있었다.하지만 기억상실은 아니다.가족도, 친구도..."
996,아르메니아 쿠니크호수에는 딱 스와코 *오미와타리 같은 자연현상이 일어난다. 호반 교...
997,"적도상 3만 6000킬로에 해당하는 우주 공간에, 길이 50M의 거대 물체가 떠있..."


In [4]:
horrorSR = horrorDF["TEXT"].str.replace(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\d ]", "", regex=True)
horrorSR.info()


<class 'pandas.core.series.Series'>
RangeIndex: 999 entries, 0 to 998
Series name: TEXT
Non-Null Count  Dtype 
--------------  ----- 
999 non-null    object
dtypes: object(1)
memory usage: 7.9+ KB


In [5]:
sentences = horrorSR.to_list()
sentences


['오늘 학교에서 모의고사보는데 가위눌리다가 한쪽눈만 떠진상태로 교실에서 내 다리에 올라와있는 귀신봤어',
 '그녀는 날 사랑한다 안 사랑한다 날 사랑한다 안 사랑한다날 사 어 더 이상 떼어낼 팔다리가 없네 넌 처음부터 날 사랑하지 않았구나',
 '어린애같지만 난 항상 지하실에서 나갈때 내 뒤를 쳐다보면서 계단을 뛰어 올라가 난 잘못된 방향을 바라보고 있었었어',
 '생리혈에 덩어리 같은게 있을수 있다는건 나도 알아 근데 그게 보통 움직이는거야',
 '난 별 생각없이 우리 모두 다함께 손뼉쳐 라고 외쳤어 다락방에서 박수 소리를 들리기 전까진 난 내가 집에 혼자있는줄 알았어',
 '난 숲속에서 일회용 카메라를 발견했다 필름을 현상했을때 그곳엔 내가 처음으로 카메라를 발견했을때의 모습이 찍힌 사진 딱 한장만이 찍혀있었다',
 '우리 할머니는 심한 치매에 걸리셨다 매일 저녁 난 할머니가 2년전에 이미 돌아가셨다고 설명하지만 허사다',
 '버스 문이 닫히고 버스 기사의 웃음소리가 그의 뒤에서 울려퍼졌을때 지미는 자신의 실수를 알아챘다 왜 스쿨 버스가 한밤중에 달리고 있는거지',
 '저녁시간 아빠는 나에게 하루가 어땠는지 물었다 아빠의 얼굴이 약간 정중앙에서 벗어나있는것처럼 보인다는 사실을 깨달았을때 난 아빠의 어깨 너머로 도망가라고 입모양으로 말하고 있는 두려움에 빠진 엄마를 볼수 있었다',
 '아빠 나 무서운 꿈 꿨어 오 공주님 난 아빠가 아냐',
 '아내는 계속 나한테 딸이 2년전에 죽었다는 사실을 납득시키려고 했다 우리가 이 이야기로 다투고 있을때 작은 목소리가 들려왔다 누구랑 얘기하고 있는거야 아빠 ',
 '수업시간에 나한테 잘해줘서 정말 고마워 내일 안전한 사람은 너뿐일거야',
 '내 스마트워치 기록이 무서워 왜 내 심장이 매일 밤 새벽 3시에 멈추는거야',
 '사람들이 날 보면 내가 엄마 눈을 가졌다고 하더라 웃긴건 말야 난 엄마 심장도 지금 가지고 있어',
 '매일 밤 잠자리에 들기전에 난 앞문이 단단히 잠겼는지 항상 확인해 마침내 오늘밤 그녀가 까먹은것

In [6]:
sentence_dot = ".".join(sentences) + "."
sentence_dot


'오늘 학교에서 모의고사보는데 가위눌리다가 한쪽눈만 떠진상태로 교실에서 내 다리에 올라와있는 귀신봤어.그녀는 날 사랑한다 안 사랑한다 날 사랑한다 안 사랑한다날 사 어 더 이상 떼어낼 팔다리가 없네 넌 처음부터 날 사랑하지 않았구나.어린애같지만 난 항상 지하실에서 나갈때 내 뒤를 쳐다보면서 계단을 뛰어 올라가 난 잘못된 방향을 바라보고 있었었어.생리혈에 덩어리 같은게 있을수 있다는건 나도 알아 근데 그게 보통 움직이는거야.난 별 생각없이 우리 모두 다함께 손뼉쳐 라고 외쳤어 다락방에서 박수 소리를 들리기 전까진 난 내가 집에 혼자있는줄 알았어.난 숲속에서 일회용 카메라를 발견했다 필름을 현상했을때 그곳엔 내가 처음으로 카메라를 발견했을때의 모습이 찍힌 사진 딱 한장만이 찍혀있었다.우리 할머니는 심한 치매에 걸리셨다 매일 저녁 난 할머니가 2년전에 이미 돌아가셨다고 설명하지만 허사다.버스 문이 닫히고 버스 기사의 웃음소리가 그의 뒤에서 울려퍼졌을때 지미는 자신의 실수를 알아챘다 왜 스쿨 버스가 한밤중에 달리고 있는거지.저녁시간 아빠는 나에게 하루가 어땠는지 물었다 아빠의 얼굴이 약간 정중앙에서 벗어나있는것처럼 보인다는 사실을 깨달았을때 난 아빠의 어깨 너머로 도망가라고 입모양으로 말하고 있는 두려움에 빠진 엄마를 볼수 있었다.아빠 나 무서운 꿈 꿨어 오 공주님 난 아빠가 아냐.아내는 계속 나한테 딸이 2년전에 죽었다는 사실을 납득시키려고 했다 우리가 이 이야기로 다투고 있을때 작은 목소리가 들려왔다 누구랑 얘기하고 있는거야 아빠 .수업시간에 나한테 잘해줘서 정말 고마워 내일 안전한 사람은 너뿐일거야.내 스마트워치 기록이 무서워 왜 내 심장이 매일 밤 새벽 3시에 멈추는거야.사람들이 날 보면 내가 엄마 눈을 가졌다고 하더라 웃긴건 말야 난 엄마 심장도 지금 가지고 있어.매일 밤 잠자리에 들기전에 난 앞문이 단단히 잠겼는지 항상 확인해 마침내 오늘밤 그녀가 까먹은것 같으니 이제 집안으로 들어갈수 있겠어 .난 오 하느님 맙소사 내가 지구의 마지막 사람이 아니었어 라고

In [7]:
char_set = list(set(sentence_dot))
char_to_id = {char: idx for idx, char in enumerate(char_set)}
id_to_char = {idx: char for idx, char in enumerate(char_set)}
dict_size = len(char_to_id)


In [8]:
X_data, y_data = [], []
sequence_length = 10

for i in range(0, len(sentence_dot) - sequence_length):
    x_str = sentence_dot[i : i + sequence_length]
    y_str = sentence_dot[i + 1 : i + sequence_length + 1]
    # print(i, x_str, '->', y_str)
    X_data.append([char_to_id[c] for c in x_str])
    y_data.append([char_to_id[c] for c in y_str])


In [9]:
X_data[0], y_data[0]


([680, 208, 475, 726, 1195, 875, 0, 475, 97, 656],
 [208, 475, 726, 1195, 875, 0, 475, 97, 656, 1070])

In [10]:
print(
    f"입력 데이터 size: {len(X_data)}, shape: {np.array(X_data).shape}, dim: {np.array(X_data).ndim}"
)
print(
    f"출력 데이터 size: {len(y_data)}, shape: {np.array(y_data).shape}, dim: {np.array(y_data).ndim}"
)


입력 데이터 size: 127551, shape: (127551, 10), dim: 2
출력 데이터 size: 127551, shape: (127551, 10), dim: 2


In [11]:
class CharDataset(Dataset):
    def __init__(
        self,
        X_data,
        y_data,
    ) -> None:
        super().__init__()
        self.in_text = torch.FloatTensor(X_data)
        self.out_text = torch.FloatTensor(y_data)

    def __len__(self):
        return len(self.out_text)

    def __getitem__(self, index):
        return self.in_text[index], self.out_text[index]


In [12]:
class CharGRU(nn.Module):
    def __init__(self, dict_size, hidden_size, n_layers, dropout) -> None:
        super().__init__()
        self.gru = nn.GRU(
            input_size=dict_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=True,
        )
        self.linear = nn.Linear(hidden_size * 2, dict_size)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()

    def forward(self, text):
        output, _ = self.gru(text)
        return self.linear(output)


In [13]:
batch_size = 128
dict_size = len(char_to_id)
hidden_size = 256
n_layer = 2
dropout = 0.2
lr = 0.001

learningDS = CharDataset(X_data, y_data)
learningDL = DataLoader(dataset=learningDS, batch_size=batch_size, shuffle=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = CharGRU(dict_size, hidden_size, n_layer, dropout).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(classifier.parameters(), lr)
schduler = optim.lr_scheduler.StepLR(optimizer, 5, 0.1)

print(len(learningDL))


997


In [14]:
from torchinfo import summary

summary(classifier)


Layer (type:depth-idx)                   Param #
CharGRU                                  --
├─GRU: 1-1                               3,506,688
├─Linear: 1-2                            643,815
Total params: 4,150,503
Trainable params: 4,150,503
Non-trainable params: 0

In [15]:
from tqdm import tqdm


def learning(
    epoch, epochs, model, dataLoader, criterion, optimizer, device, mode="train"
):
    if mode == "train":
        model.train()
        is_train = True
    elif mode == "test":
        model.eval()
        is_train = False
    else:
        raise ValueError("Invalid mode. Must be 'train' or 'test'")

    loss_list = []
    acc_list = []
    f1_list = []
    running_loss = 0.0

    with torch.set_grad_enabled(is_train):
        pbar = tqdm(enumerate(dataLoader), total=len(dataLoader))
        for step, (in_text, out_text) in pbar:
            in_text, out_text = in_text.to(device), out_text.to(device)
            oh_in_text = F.one_hot(in_text.long(), dict_size).float()
            oh_out_text = F.one_hot(out_text.long(), dict_size).float()
            pre_text = model(oh_in_text)
            loss = criterion(pre_text, oh_out_text)
            running_loss += loss.item()
            acc_list.append(
                accuracy(
                    pre_text.argmax(2),
                    out_text,
                    task="multiclass",
                    num_classes=dict_size,
                )
            )
            f1_list.append(
                f1_score(
                    pre_text.argmax(2),
                    out_text,
                    task="multiclass",
                    num_classes=dict_size,
                    average="macro",
                )
            )
            if is_train:
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
                schduler.step()
                optimizer.step()
                if step % 10 == 9:
                    pbar.set_description(
                        f"Epoch [{epoch + 1}/{epochs}], 횟수 [{step + 1}/{len(dataLoader)}], Loss: {running_loss:.4f}"
                    )
                    loss_list.append(running_loss)
                    running_loss = 0.0
    return np.mean(loss_list), np.mean(acc_list), np.mean(f1_list)


In [16]:
def predict(model, word):
    pred_text = []
    for char in word:
        if char in char_to_id.keys():
            pred_text.append(
                F.one_hot(torch.tensor(char_to_id[char]), dict_size).float()
            )
        else:
            pred_text.append(
                F.one_hot(torch.tensor(char_to_id[" "]), dict_size).float()
            )
    pred_text = torch.stack(pred_text).to(device)
    model.eval()
    with torch.no_grad():
        pred_num = model(pred_text).argmax(1).tolist()
        result = "".join([id_to_char[i] for i in pred_num])
    return result


In [17]:
def make_sentence(model, start, num=100, stopword="."):
    sentence = start
    for _ in range(num):
        # print(f"{_}", sentence)
        # next_char = predict(classifier, start)
        next_char = predict(model, sentence[-10:])
        # print("==>", next_char[-1])
        # start = next_char
        sentence += next_char[-1]
        if next_char[-1] == stopword:
            break
    return sentence


In [18]:
scoreList = [[], [], []]
resultList = []


In [20]:
epochs = 1000
for epoch in range(epochs):
    loss, acc, f1 = learning(
        epoch,
        epochs,
        classifier,
        learningDL,
        criterion,
        optimizer,
        device,
        mode="train",
    )
    resultList.append(make_sentence(classifier, "오늘 학교에서 모의"))
    scoreList[0].append(loss), scoreList[1].append(acc), scoreList[2].append(f1)
    torch.save(classifier, "classifier.pt")


Epoch [1/1000], 횟수 [120/997], Loss: 0.1718:  12%|█▏        | 120/997 [00:30<03:41,  3.95it/s]


KeyboardInterrupt: 

In [ ]:
make_sentence(classifier, "괴담 생성", num=1000)
